<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/final_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import package
!pip install censusgeocode
!pip install rets
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np
import sys

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import censusgeocode as cg
import numpy as np
from rets import Session
def findifNull(lo_left, lo_right, la_left, la_right, research, zip):
    result = research.search(resource='Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-05-31), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (PostalCode = {})'.format(lo_left, lo_right, la_left, la_right, zip))
    count = 0
    lis = []
    di = {}
    df = pd.DataFrame()
    for item in result:
        if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True)
            count += 1
            di = item
    for key in di:
        lis.append(key)
    if(count >= 5):
        df = df.T
        df.columns = lis
        return df
    else:
        lo_left -= 0.008
        lo_right += 0.008
        la_left -= 0.008
        la_right += 0.008
        return findifNull(lo_left, lo_right, la_left, la_right, research, zip)
def nullZiphouse(house_leftLa,house_rightLa,house_leftLo,house_rightLo,research,column):
  result = research.search(resource = 'Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-06-30), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-)'.format(house_leftLo, house_rightLo, house_leftLa, house_rightLa))
  count = 0
  df = pd.DataFrame()
  for item in result: 
    if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) #?
            count += 1         
  if(count >= 5):  
    df = df.T
    df.columns = column # 
    return df
  else:
    house_leftLo -= 0.008
    house_rightLo += 0.008
    house_leftLa -= 0.008
    house_rightLa += 0.008
    return nullZiphouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, research, column)
    
def findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, research, column, zip, a, b, time1, time2):
    basic = house_leftSqt
    count = 0
    df = pd.DataFrame()
    result = research.search(resource='Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-05-31), (YearBuilt = {}+), (YearBuilt = {}-), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (LivingArea = {}+), (LivingArea = {}-), (PostalCode = {})'.format(time1, time2, house_leftLo, house_rightLo, house_leftLa, house_rightLa, house_leftSqt, house_rightSqt, zip))
    for item in result:
        if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True)
            count += 1
    if(count >= 5):
        df = df.T
        df.columns = column
        return df
    else:
        house_leftLa -= 0.008
        house_rightLa += 0.008
        house_leftLo -= 0.008
        house_rightLo += 0.008
        b +=1
        if(b >= 12):
          house_leftLo += 0.096
          house_rightLo -= 0.096
          house_leftLa += 0.096
          house_rightLa -= 0.096  
          return nullZiphouse(house_leftLa,house_rightLa,house_leftLo,house_rightLo,research,column)

  #Living Area
        if(a == 0):
            if(basic <= 1100):
                house_leftSqt -= 100
                house_rightSqt += 100
            elif(basic <= 2200):
                house_leftSqt -= 200
                house_rightSqt += 200
            else:
                house_leftSqt -= 300
                house_rightSqt += 300

            a += 1
        return findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, research, column, zip, a, b, time1, time2)

    
def findfirst(result):
    count = 0
    for item in result:
        count += 1
    if(count == 0):
        return False
    else:
        return True    
    
def insertId(pulled_value, mls, rets_client):
    
    column = []
    for key in pulled_value:
        column.append(key)
    print("\nThis value exists in Website")
  # get whole row value
  #get each value
    zip = pulled_value.get('PostalCode')
    house_leftLa = float(pulled_value.get('Latitude'))
    house_rightLa = float(pulled_value.get('Latitude')) 
    house_leftLo = float(pulled_value.get('Longitude'))
    house_rightLo = float(pulled_value.get('Longitude'))
    house_leftSqt = float(pulled_value.get('LivingArea'))
    house_rightSqt = float(pulled_value.get('LivingArea'))
    time1 = int(pulled_value.get('YearBuilt'))
    time2 = int(pulled_value.get('YearBuilt'))
    if time1 < 1990:
        time1 = 0 
        time2 = 1989
    elif time1 >=1990 and time1 <2011:
        time1 = 1990
        time2 = 2010
    elif time1 >= 2011 and time1 < 2022:
        time1 = 2011
        time2 = 2021
    else:
        time1 = 2022
        time2 = 9999
    a = 0
    b = 0
    result1 = findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, rets_client, column, zip, a, b, time1, time2)        
    result1 = result1[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
    price = []
    for item in result1['ClosePrice']:
        item = float(item)
        price.append(item)
    result1['ClosePrice'] = price
    mean1 = result1['ClosePrice'].mean()
    top1 = list(result1['ClosePrice'])
    top1.sort(reverse = True)
    print('The mean for {} is {}'.format(mls, mean1))
    print('The top 3 for {} are {}'.format(mls, top1[0:3]))
    return result1.to_csv('{}.csv'.format(mls))
def findAdd(address, city, state, zip, rets_client):
    
    result = cg.address(address, city = city, state= state,zipcode = zip)
    lo_left = result[0]['coordinates']['x']
    lo_right = result[0]['coordinates']['x']
    la_left = result[0]['coordinates']['y']
    la_right = result[0]['coordinates']['y']
    result = findifNull(lo_left, lo_right, la_left, la_right, rets_client, zip)
    result = result[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
    price = []
    for item in result['ClosePrice']:
        item = float(item)
        price.append(item)
    result['ClosePrice'] = price
    mean = result['ClosePrice'].mean()
    top = list(result['ClosePrice'])
    top.sort(reverse = True)
    print('The mean for {}, {}, {} is {}'.format(address, state, zip, mean))
    print('The top 3 for {}, {}, {} are {}'.format(address, state, zip, top))
    return result.to_csv('{},{},{}.csv'.format(address, state, zip))
    
def get(number, finalN, code, i, rets_client, street, city, state):
    pulled_value = {}
    result = rets_client.search(resource='Property', resource_class='Property', dmql_query='(StreetNumber = {}),(StreetName = {}),(PostalCode = {})'.format(number[i],finalN[i],code[i]))
    for item in result:
        pulled_value = item
    if(len(pulled_value) != 0):
        column = []
        for key in pulled_value:
            column.append(key)
        print("\nThis value exists in Website")
  # get whole row value
  #get each value
        zip = pulled_value.get('PostalCode')
        house_leftLa = float(pulled_value.get('Latitude'))
        house_rightLa = float(pulled_value.get('Latitude')) 
        house_leftLo = float(pulled_value.get('Longitude'))
        house_rightLo = float(pulled_value.get('Longitude'))
        house_leftSqt = float(pulled_value.get('LivingArea'))
        house_rightSqt = float(pulled_value.get('LivingArea'))
        time1 = int(pulled_value.get('YearBuilt'))
        time2 = int(pulled_value.get('YearBuilt'))
        if time1 < 1990:
            time1 = 0 
            time2 = 1989
        elif time1 >=1990 and time1 <2011:
            time1 = 1990
            time2 = 2010
        elif time1 >= 2011 and time1 < 2022:
            time1 = 2011
            time2 = 2021
        else:
            time1 = 2022
            time2 = 9999
        a = 0
        b = 0
        result1 = findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, rets_client, column, zip, a, b, time1, time2)
        result1 = result1[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
        price = []
        for item in result1['ClosePrice']:
            item = float(item)
            price.append(item)
        result1['ClosePrice'] = price
        mean1 = result1['ClosePrice'].mean()
        top1 = list(result1['ClosePrice'])
        top1.sort(reverse = True)
        return result1.to_csv('{} {} {}.csv'.format(number[i],finalN[i],code[i])), mean1, top1[0:3]
    else:
        print('check coordinates')
        result = cg.address(street[i], city = city[i], state= state[i], zipcode = code[i])
        if(len(result) != 0):
            lo_left = result[0]['coordinates']['x']
            lo_right = result[0]['coordinates']['x']
            la_left = result[0]['coordinates']['y']
            la_right = result[0]['coordinates']['y']
            result = findifNull(lo_left, lo_right, la_left, la_right, rets_client, code[i])
            result = result[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
            price = []
            for item in result['ClosePrice']:
                item = float(item)
                price.append(item)
            result['ClosePrice'] = price
            mean = result['ClosePrice'].mean()
            top = list(result['ClosePrice'])
            top.sort(reverse = True)
            return result.to_csv('{} {} {}.csv'.format(number[i],finalN[i],code[i])), mean, top[0:3]
        else:
            print('invalid address')
            mean = 0
            top = 0
            return 0, mean, top
def judge(i):
    if i % 10 == 1:
        return 'st'
    elif i % 10 == 2:
        return 'nd'
    elif i % 10 == 3:
        return 'rd'
    else:
        return 'th'
def main():
    login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
    username = '0671181_NID'
    password = 'Rt$tg6jx'
    rets_client = Session(login_url, username, password)
    rets_client.login()
    while True:
        print('You want to insert values or a csv_address, csv_mls, please either insert csv or value!')
        choice = input()
        if choice == 'csv_address':
            while True:
                print('select which csv you want to use')
                csv = input()
                try:
                    data = pd.read_csv('{}.csv'.format(csv))        
                    code = []
                    name = []
                    number = []
                    street = []
                    city = []
                    state = []
                    for item in data['FullAdress']:
                        a = item.split(',')[0]
                        b = item.split(',')[2]
                        c = item.split(',')[1]
                        street.append(a)
                        city.append(c)
                        numbers = a.split()[0]
                        names = a.split()[1:(len(a.split()) - 1)]
                        codes = b.split()[1][0:5]
                        states = b.split()[0]
                        state.append(states)
                        code.append(codes)
                        name.append(names)
                        number.append(numbers)
                    finalN = []
                    for item in name:
                        string = ""
                        for items in item:
                            string += items
                            string += ' '
                        finalN.append(string[0:len(string) - 1])            
                    for i in range(len(code)):
                        result, mean, top = get(number, finalN, code, i, rets_client, street, city, state)
                        print('The mean for {}, {}, {} is {}'.format(number[i],finalN[i],code[i], mean))
                        print('The top 3 for {}, {}, {} are {}'.format(number[i],finalN[i],code[i], top))
                    return
                except FileNotFoundError:
                    print("Oops! There is no file!  Try again...")
        elif(choice =='csv_mls'):# my new code
          while True:
            print("Which csv do you want to use? (make sure you have listingId)")
            csv = input()
            try:
              lis_mls = []
              data = pd.read_csv('{}.csv'.format(csv)) 
              for item in data['ListingId']:
                lis_mls.append(int(item))
              for i in range(len(lis_mls)):
                pulled_value = {}
                result = rets_client.search(resource='Property', resource_class='Property', dmql_query='(ListingId = {})'.format(lis_mls[i]))
                for items in result:
                  pulled_value = items
                if(len(pulled_value) != 0):
                  insertId(pulled_value, lis_mls[i], rets_client) 
                  if(i == len(lis_mls) - 1):
                    return #end here

            except FileNotFoundError:
                    print("Oops! There is no file!  Try again...") 

        elif(choice == 'value'):
            while True:
                print('How many mls you want to check?')
                number = input()
                try:
                    Id_list = []
                    for i in range(int(number)):
                        print('Enter the {}{} #MLS'.format(i+1, judge(i+1)))
                        mls = input()
                        Id_list.append(mls)
            #14730621

                    for i in range(len(Id_list)):
                        pulled_value = {}
                        result = rets_client.search(resource='Property', resource_class='Property', dmql_query='(ListingId = {})'.format(Id_list[i]))
                        for items in result:
                            pulled_value = items
                        if(len(pulled_value) != 0):
                            insertId(pulled_value, Id_list[i], rets_client) 
                            if(i == int(number) - 1):
                                return
                        else:

                            print('Id {} not exists, please insert specific value'.format(Id_list[i]))
                            print("If you don't know the address, please insert skip")
                            deci = input()
                            if(deci == 'skip'):
                                if(i == int(number) - 1):
                                    return
                                continue
                                
                            else:
                                print('Enter address')
                                address = input()
        #'8615 Barclay Street'
                            
                                print('Enter city')
                                city = input()
        #'Dallas'
                                print('Enter state')
                                state = input()
        #'Texas'
                                print('Enter zipcode')
                                zip = input()
        #'75227'
                                try:
                                    findAdd(address, city, state, zip, rets_client)
                                    if(i == int(number) - 1):
                                        return
                                except IndexError:
                                    print("invalid input, please next value!")
                                    if(i == int(number) - 1):
                                        return                            
                except ValueError:
                    print("invalid input, try again!")
        else:
            print("invalid input, try again!")
            
if __name__ == "__main__":
    print(main())

You want to insert values or a csv_address, csv_mls, please either insert csv or value!
csv_mls
Which csv do you want to use? (make sure you have listingId)
BOM(8)

This value exists in Website
The mean for 20008859 is 344857.14285714284
The top 3 for 20008859 are [435000.0, 380000.0, 345000.0]

This value exists in Website
The mean for 20028359 is 115428.57142857143
The top 3 for 20028359 are [235000.0, 170000.0, 155000.0]

This value exists in Website
The mean for 20031838 is 445512.5
The top 3 for 20031838 are [529000.0, 525000.0, 462000.0]

This value exists in Website
The mean for 20046864 is 448211.1111111111
The top 3 for 20046864 are [600000.0, 490000.0, 470000.0]

This value exists in Website
The mean for 20064017 is 337263.3333333333
The top 3 for 20064017 are [363080.0, 345000.0, 340000.0]

This value exists in Website
The mean for 20069995 is 230580.0
The top 3 for 20069995 are [273000.0, 230000.0, 220000.0]

This value exists in Website
The mean for 20076957 is 318678.3333

# My code

In [ ]:
import pandas as pd
import censusgeocode as cg
import numpy as np
from rets import Session

# select the data columns we want 

def findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, research, column, zip):
    basic = house_leftSqt
    count = 0
    df = pd.DataFrame()
    result = research.search(resource='Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-05-31), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (LivingArea = {}+), (LivingArea = {}-), (PostalCode = {})'.format(house_leftLo, house_rightLo, house_leftLa, house_rightLa, house_leftSqt, house_rightSqt, zip))
    for item in result:
        if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True)
            count += 1
    if(count >= 5):
        df = df.T
        df.columns = column
        return df
    else:
        ######## 改成0.008 ########(
        house_leftLa -= 0.008
        house_rightLa += 0.008
        house_leftLo -= 0.008
        house_rightLo += 0.008

        return findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, research, column, zip)  
def findifNull(lo_left, lo_right, la_left, la_right, research, zip):
    result = research.search(resource='Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-05-31), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (PostalCode = {})'.format(lo_left, lo_right, la_left, la_right, zip))
    count = 0
    lis = []
    di = {}
    df = pd.DataFrame()
    for item in result:
        if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True)
            count += 1
            di = item
    for key in di:
        lis.append(key)
    if(count >= 5):
        df = df.T
        df.columns = lis
        return df
    else:
        lo_left -= 0.01
        lo_right += 0.01
        la_left -= 0.01
        la_right += 0.01
        return findifNull(lo_left, lo_right, la_left, la_right, research, zip)
login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
username = '0671181_NID'
password = 'Rt$tg6jx'
rets_client = Session(login_url, username, password)
rets_client.login()
def findfirst(result):
    count = 0
    for item in result:
        count += 1
    if(count == 0):
        return False
    else:
        return True

def main():
    print('Enter #MLS')
    mls = input()
    #14730621
    pulled_value = {}
    result = rets_client.search(resource='Property', resource_class='Property', dmql_query='(ListingId = {})'.format(mls))
    for item in result:
        pulled_value = item
    if(len(pulled_value) != 0):
        column = []
        for key in pulled_value:
            column.append(key)
        print("\nThis value exists in Website")       
  # get whole row value
  #get each value
        zip = pulled_value.get('PostalCode')
        house_leftLa = float(pulled_value.get('Latitude'))
        house_rightLa = float(pulled_value.get('Latitude')) 
        house_leftLo = float(pulled_value.get('Longitude'))
        house_rightLo = float(pulled_value.get('Longitude'))
        
        ######## Moved the basic and house left&right here ########
        basic = float(pulled_value.get('LivingArea'))
        #print(basic)
        house_leftSqt = ''
        house_rightSqt =''
        #house_leftSqt = float(pulled_value.get('LivingArea'))
        #house_rightSqt = float(pulled_value.get('LivingArea'))
        if(basic <= 1100):
            house_leftSqt = basic - 100
            house_rightSqt = basic + 100
        elif(basic <= 2200):
            house_leftSqt = basic - 200
            house_rightSqt = basic + 200
        else:
            house_leftSqt = basic - 300
            house_rightSqt = basic + 300
        
        result = findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, rets_client, column, zip)
        result = result[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
        price = []
        for item in result['ClosePrice']:
            item = float(item)
            price.append(item)
        result['ClosePrice'] = price
        mean = result['ClosePrice'].mean()
        top = list(result['ClosePrice'])
        top.sort(reverse = True)
        return result.to_csv('result.csv'), mean, top[0:3]
    #.to_csv('result.csv'), mean, top[0:3]
    else:
        print('invalid ID, please insert specific value')
        print('Enter address')
        address = input()
        #'8615 Barclay Street'
        print('Enter city')
        city = input()
        #'Dallas'
        print('Enter state')
        state = input()
        #'Texas'
        print('Enter zipcode')
        zip = input()
        #'75227'
        result = cg.address(address, city = city, state= state,zipcode = zip)
        lo_left = result[0]['coordinates']['x']
        lo_right = result[0]['coordinates']['x']
        la_left = result[0]['coordinates']['y']
        la_right = result[0]['coordinates']['y']
        result = findifNull(lo_left, lo_right, la_left, la_right, rets_client, zip)
        result = result[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
        price = []
        for item in result['ClosePrice']:
            item = float(item)
            price.append(item)
        result['ClosePrice'] = price
        mean = result['ClosePrice'].mean()
        top = list(result['ClosePrice'])
        top.sort(reverse = True)
        return result.to_csv('result.csv'), mean, top[0:3]
  # # limit zipcode range
    
if __name__ == "__main__":
    print(main())